Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# 04 Forecasting Pipeline

In this notebook we create a pipeline for Forcasting 11,973 ARIMA models. The training and scoring of these models was completed in the Training and Scoring notebooks in this repository. We will set up the Pipeline for forecasting given the desired forecasting horizon. We utitlize the [ParallelRunStep](https://docs.microsoft.com/en-us/python/api/azureml-contrib-pipeline-steps/azureml.contrib.pipeline.steps.parallel_run_step.parallelrunstep?view=azure-ml-py) to parallelize the process. For more information about the Data and Models refer to the Data Preparation and Training Notebooks. 

The pipeline set up is similar to the Training Pipeline in this repository. For more details on the steps and functions refer to the Training folder. 

## Prerequisites 

This example runs on an Azure Machine Learning Notebook VM. We are calling models that have **already been trained and registered** to the Workspace. If you have already run the Environment Setup and Training Pipeline notebooks or you have an AML Notebook set up with Models registered to the Workspace you are all set. 

## 1.0 Call the Workspace, Datastore, and Compute

As we did in the Training Pipeline notebook, we need to call the Workspace. We also want to create variables for the datastore and compute cluster. 

### Connect to the workspace


In [ ]:
from azureml.core import Workspace 

ws= Workspace.from_config(path='../aml_config/config.json')

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

### Attach existing compute resource


In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget

compute = AmlCompute(ws, 'train-many-model')

### Set up an Experiment

In [ ]:
from azureml.core import Experiment

experiment = Experiment(ws, 'forecasting-pipeline')

### Call the Datastore

In [ ]:
from azureml.core import Datastore

dstore = ws.get_default_datastore()
forecast_output_dstore = Datastore(ws,'forecasting_output_datastore')

## 2.0 Call Registered FileDataset
In the Data Preparation notebook, we registered the orange juice data to the Workspace. You can choose to run the pipeline on the subet of 10 series or the full dataset of 11,973 series. We recommend starting with 10 series then expanding. 

In [ ]:
from azureml.core.dataset import Dataset

filedst_10_models = Dataset.get_by_name(ws, name='oj_data_small')
filedst_10_models_input = filedst_10_models.as_named_input('forecast_10_models')
 
filedst_all_models = Dataset.get_by_name(ws, name='oj_data')
filedst_all_models_input = filedst_all_models.as_named_input('forecast_all_models')

## 3.0 Build forecasting pipeline
Now that the data, models, and compute resources are set up, we can put together a pipeline for forecasting. 
### Set up the environment to run the script
Specify the conda dependencies for your script. This will allow us to install packages and configure the environment.

In [ ]:
from azureml.core import Environment
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.conda_dependencies import CondaDependencies

batch_conda_deps = CondaDependencies.create(pip_packages=['sklearn','pmdarima'])
batch_env = Environment(name="manymodels_environment")
batch_env.python.conda_dependencies = batch_conda_deps
batch_env.docker.enabled = True
batch_env.docker.base_image = DEFAULT_CPU_IMAGE

### Create the configuration to wrap the entry script 
In the [ParallelRunConfig](https://docs.microsoft.com/en-us/python/api/azureml-contrib-pipeline-steps/azureml.contrib.pipeline.steps.parallelrunconfig?view=azure-ml-py), we will call the entry script, environment configuration, and parameters. You will want to determine the number of workers and nodes appropriate for your use case. We use the same settings determined in the Training Notebook. Refer to the Tutorial section to create a custom entry_script. 

In [ ]:
from azureml.contrib.pipeline.steps import ParallelRunStep, ParallelRunConfig 

workercount = 8
nodecount = 5
timeout = 500

tags1 = {}
tags1['nodes'] = nodecount
tags1['workers-per-node'] = workercount
tags1['timeout'] = timeout 

parallel_run_config = ParallelRunConfig(
    source_directory = './scripts',
    entry_script = 'forecast.py',
    mini_batch_size = '1',
    run_invocation_timeout = timeout, 
    error_threshold = 10,
    output_action = 'append_row', 
    environment = batch_env, 
    process_count_per_node = workercount, 
    compute_target = compute, 
    node_count = nodecount
)

### Create the ParallelRunStep
 The [ParallelRunStep](https://docs.microsoft.com/en-us/python/api/azureml-contrib-pipeline-steps/azureml.contrib.pipeline.steps.parallel_run_step.parallelrunstep?view=azure-ml-py) is the main step in our pipeline. We specified the following parameters: **input**, **output**, and **arguments**. We also set the output directory.   

For the orange juice sales forecasting, we pass two **arguments** to the entry_script. 
- **forecast_horizon** is how far into the future the forecast should extend.
- **starting_date** is the date to begin forecating. 

*arguments* and *inputs* are the two parameters that can pass information to the entry_script.

You can change between running the pipeline on a subset of models or the full data set by changing the inputs parameter. 

In [ ]:
from azureml.pipeline.core import PipelineData

datasetname = 'store'
output_dir = PipelineData(name = 'forecasting_output', 
                         datastore = dstore)

parallelrun_step = ParallelRunStep(
    name="many-models-forecasting",
    parallel_run_config=parallel_run_config,
    inputs=[filedst_10_models_input], 
    #inputs=[filedst_all_models_input],
    output=output_dir,
    models= [], 
    arguments=['--forecast_horizon', 8,
              '--starting_date', '1992-10-01'])

### Create Output Step
We create a [PythonScriptStep](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps.python_script_step.pythonscriptstep) as a second step in the pipeline to retrieve the output from the ParallelRunStep and upload the results to a specificed path in Blob storage. 

### Set up RunConfiguration

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.runconfig import CondaDependencies

conda_run_config = RunConfiguration(framework="python")
conda_run_config.target = compute
conda_run_config.environment.docker.enabled = True
conda_run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE
cd = CondaDependencies.create(pip_packages=['azureml-pipeline-core'], conda_packages=['pandas'])
conda_run_config.environment.python.conda_dependencies = cd

### Create PythonScriptStep

In [ ]:
from azureml.pipeline.steps import PythonScriptStep

upload_predictions_step = PythonScriptStep(name="upload_predictions",
                        script_name="upload_predictions.py",
                        compute_target=compute,
                        source_directory='./scripts',
                        runconfig=conda_run_config,
                        arguments=['--parallelrunstep_name',parallelrun_step.name, 
                                   '--pipeline_output_name', output_dir.name, 
                                   '--datastore', forecast_output_dstore.name, 
                                   '--experiment', experiment.name, 
                                   '--overwrite_predictions', True])

### Set up step sequence  
We set up a step sequence to ensure the ParallelRunStep executes before the PythonScriptStep. We need to ParallelRunStep to fully complete to collect the logs from each run and output them to the specificed directory. 

In [ ]:
from azureml.pipeline.core import StepSequence

forecasting_steps = StepSequence(steps=[parallelrun_step, upload_predictions_step])

## 4.0 Run the forecast pipeline
We can use the Experiment we created to track the runs of the pipeline and review the output. With the current settings and the Standard_D13_V2 VM the pipeline takes approximately 1h 5m to run all 11,973 models.  

In [ ]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace = ws, steps=forecasting_steps)
run = experiment.submit(pipeline, tags=tags1)

## 5.0 Publish the pipeline
After successfully setting up the pipeline, we publish the pipeline to the Workspace. [Published pipelines](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.publishedpipeline?view=azure-ml-py) create an endpoint to call the pipeline to run without having to open the code used to create it. It can be used to resubmit the pipline with different parameter inputs. 

In [ ]:
published_pipeline = pipeline.publish(name = 'forecast-models',
                                     description = 'forecast models and log the run',
                                     version = '1',
                                     continue_on_step_failure = False)

## 6.0 Schedule the pipeline weekly
We can use the pipeline id to scheudle the pipeline to run at a specified interval. We schedule our forecasting pipeline to run weekly. 

In [ ]:
from azureml.pipeline.core import Schedule, ScheduleRecurrence
    
forecast_pipeline_id = published_pipeline.id

recurrence = ScheduleRecurrence(frequency="Week", interval=1, start_time="2020-01-01T13:00:00")
recurring_schedule = Schedule.create(ws, name="Forecasting-Pipeline-Recurring-Schedule", 
                            description="Schedule forecasting Pipeline to run on the first day of every month starting Jan 1, 2020 at 1PM",
                            pipeline_id=forecast_pipeline_id, 
                            experiment_name=experiment.name, 
                            recurrence=recurrence)

## 7.0 Pipeline Outputs
The forecasting pipeline forecasts the orange juice quantity for a Store by Brand for the next 8 weeks. The pipeline returns one file with the predictions for each store and outputs the result to the forecasting_output Blob container. 